In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_needle(image, center_x, center_y, radius):
    """
    Phát hiện kim trong ảnh thông qua phân đoạn cạnh và Hough Transform.
    
    Parameters:
    - image: Hình ảnh đầu vào.
    - center_x, center_y: Tọa độ trung tâm của đồng hồ.
    - radius: Bán kính của đồng hồ (vùng chứa kim).

    Returns:
    - segmented_edges: Hình ảnh phân đoạn cạnh.
    - lines: Các đường thẳng phát hiện được (được coi là kim đồng hồ).
    """
    # Chuyển đổi ảnh sang xám
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Làm mờ ảnh để giảm nhiễu
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Phát hiện cạnh bằng Canny
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)
    
    # Tạo mặt nạ chỉ quan tâm đến khu vực quanh tâm với bán kính nhất định
    mask = np.zeros_like(edges)
    cv2.circle(mask, (center_x, center_y), radius, 255, -1)  # Vùng bán kính
    segmented_edges = cv2.bitwise_and(edges, mask)  # Áp dụng mặt nạ lên ảnh cạnh

    # Phát hiện đường thẳng trong khu vực đã phân đoạn bằng Hough Transform
    lines = cv2.HoughLinesP(segmented_edges, rho=1, theta=np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)
    
    return segmented_edges, lines


def calculate_angle(center_x, center_y, x1, y1, x2, y2):
    """
    Tính toán góc của kim đồng hồ từ các điểm (x1, y1) và (x2, y2).
    
    Parameters:
    - center_x, center_y: Tọa độ trung tâm của đồng hồ.
    - x1, y1, x2, y2: Tọa độ của hai điểm trên đường thẳng (các điểm của kim).

    Returns:
    - angle: Góc tính được tính từ trục hoành (theo chiều kim đồng hồ).
    """
    dist1_sq = (x1 - center_x)**2 + (y1 - center_y)**2
    dist2_sq = (x2 - center_x)**2 + (y2 - center_y)**2
    if dist1_sq > dist2_sq:
        x_tip, y_tip = x1, y1
    else:
        x_tip, y_tip = x2, y2
    dx = x_tip - center_x
    dy = center_y - y_tip
    angle = np.degrees(np.arctan2(dy, dx))
    return angle + 360 if angle < 0 else angle


def map_angle_to_value(angle, min_angle, max_angle, min_value, max_value):
    """
    Chuyển đổi góc thành giá trị trong phạm vi đồng hồ.
    
    Parameters:
    - angle: Góc của kim.
    - min_angle, max_angle: Góc tương ứng với giá trị min và max của đồng hồ.
    - min_value, max_value: Giá trị min và max của đồng hồ đo.

    Returns:
    - value: Giá trị đo tương ứng với góc.
    """
    if angle < min_angle:
        value = min_value + (max_value - min_value) * (min_angle - angle) / 270
    elif angle > max_angle:
        value = min_value + (max_value - min_value) * (angle - max_angle) / 270
    else:
        value = None
    return value


def process_image(image_path, center_x, center_y, radius, min_angle, max_angle, min_value, max_value):
    """
    Xử lý ảnh, phát hiện kim và tính toán giá trị đồng hồ.
    
    Parameters:
    - image_path: Đường dẫn đến ảnh cần xử lý.
    - center_x, center_y: Tọa độ trung tâm của đồng hồ.
    - radius: Bán kính của đồng hồ.
    - min_angle, max_angle: Góc tương ứng với giá trị min và max của đồng hồ.
    - min_value, max_value: Giá trị min và max của đồng hồ đo.
    """
    # Đọc ảnh
    image = cv2.imread(image_path)

    # Phát hiện kim và phân đoạn cạnh
    segmented_edges, lines = detect_needle(image, center_x, center_y, radius)
    output_frame = image.copy()

    # Tính toán góc của kim đồng hồ
    needle_angle = None
    if lines is not None:
        for line in lines:
            # Mỗi line là một mảng [x1, y1, x2, y2]
            x1, y1, x2, y2 = line[0]
            cv2.line(output_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Vẽ đường thẳng kim đồng hồ
            needle_angle = calculate_angle(center_x, center_y, x1, y1, x2, y2)
            break  # Giả sử chỉ có một kim, lấy đường thẳng đầu tiên

    # Chuyển góc thành giá trị đồng hồ
    gauge_value = None
    if needle_angle is not None:
        gauge_value = map_angle_to_value(needle_angle, min_angle, max_angle, min_value, max_value)

    # Vẽ hình tròn và tâm đồng hồ
    cv2.circle(output_frame, (center_x, center_y), radius, (0, 255, 0), 2)
    cv2.circle(output_frame, (center_x, center_y), 3, (0, 0, 255), -1)

    # Hiển thị giá trị đồng hồ
    if gauge_value is not None:
        cv2.putText(output_frame, f"Value: {gauge_value:.2f}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Hiển thị kết quả
    cv2.imshow("Processed Image", output_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Cấu hình đồng hồ đo
center_x, center_y = 279, 331  # Tọa độ trung tâm của đồng hồ
radius = 210  # Bán kính đồng hồ
min_angle = 225  # Góc min (0 bar)
max_angle = 315  # Góc max (4 bar)
min_value = 0    # Giá trị min
max_value = 4    # Giá trị max

image_path = "frame_0203.jpg"  # Đường dẫn đến ảnh cần xử lý

# Xử lý ảnh
process_image(image_path, center_x, center_y, radius, min_angle, max_angle, min_value, max_value)
